In [ ]:
from anndata import AnnData
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import leidenalg

from numpy.random import default_rng

import matplotlib.pyplot as plt

from matplotlib.pyplot import figure

figure(figsize=(20, 20), dpi=80)

In [ ]:
re=pd.read_excel("/mnt/c/Users/Sasha/Desktop/test/SCRINSHOT-Healthy-Atlas-all-samples.xlsx")

In [ ]:
re.columns

In [ ]:
re

In [ ]:
#Filter cells for more than 25 marker dots
resub=re[re.iloc[:,14:68].sum(axis=1)>25]

In [ ]:
resub

In [ ]:
#define metadata and data of interest (genes of all datasets)
metadata=resub.iloc[:,0:14]
expdata=resub.iloc[:,14:68]

In [ ]:
expdata

In [ ]:
metadata

In [ ]:
adata = sc.AnnData(expdata)
adata.obs=metadata

In [ ]:
adata.obs_names = pd.RangeIndex(stop=adata.n_obs)
adata

In [ ]:
adataraw=adata

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
adatanorm=adata

In [ ]:
expdata_norm=(adatanorm)

In [ ]:
adatanorm

In [ ]:
adatanorm.obs=metadata

In [ ]:
adatanorm.obs_names = pd.RangeIndex(stop=adatanorm.n_obs)
adatanorm

In [ ]:
sc.pp.neighbors(adatanorm, n_neighbors=20, n_pcs=7)
sc.tl.leiden(adatanorm,resolution=1.5,key_added='clusters-pc7')

In [ ]:
adatanorm.obs['clusters-pc7'].unique()

In [ ]:
sc.tl.paga(adatanorm, groups='clusters-pc7')
sc.pl.paga(adatanorm, color=['clusters-pc7'])

In [ ]:
sc.tl.umap(adatanorm,min_dist=0.3)#,init_pos='paga')
sc.set_figure_params(scanpy=True, dpi=200)

In [ ]:
#the following plots are to visualize the data
sc.pl.umap(adatanorm,color=['clusters-pc7'],cmap='viridis')#,save='.svg')

In [ ]:
sc.pl.umap(adatanorm,color=adata.var.index[0:75],cmap='viridis',vmax=10)

In [ ]:
sc.pl.umap(adatanorm,color=adata.var.index[0:75],cmap='viridis',vmax=20)

In [ ]:
sc.pl.umap(adatanorm,color=['Region'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Sample'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Position'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Donor'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Cell.class'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Cell.type'], s=1)

In [ ]:
sc.pl.umap(adatanorm,color=['Cell.subtype'], s=1)

In [ ]:
#to check if clustering is affected by sample
sc.tl.rank_genes_groups(adatanorm,groupby='Sample')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adatanorm)

In [ ]:
sc.pl.rank_genes_groups(adatanorm)

In [ ]:
#to check if clustering is affected by donor
sc.tl.rank_genes_groups(adatanorm,groupby='Donor')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adatanorm)

In [ ]:
sc.pl.rank_genes_groups(adatanorm)

In [ ]:
#to define differentially expressed genes by cluster
sc.tl.rank_genes_groups(adatanorm, groupby='clusters-pc7', method='wilcoxon')
sc.tl.dendrogram(adatanorm,groupby='clusters-pc7')
sc.pl.rank_genes_groups_dotplot(adatanorm, n_genes=5, swap_axes=True)#,save='deg.pdf')

In [ ]:
#for pre-defined cell types only
sc.tl.rank_genes_groups(adatanorm, groupby='Cell.type', method='wilcoxon')
sc.tl.dendrogram(adatanorm,groupby='Cell.type')
sc.pl.rank_genes_groups_dotplot(adatanorm, n_genes=5, swap_axes=True)#,save='deg.pdf')

In [ ]:
writer = pd.ExcelWriter('All-SCRINSHOT-clusters-pc7-upd.xlsx')
# write dataframe to excel
adatanorm.obs.to_excel(writer)
# save the excel
writer.save()

In [ ]:
#to visulaize clusters by position
colors=dict(zip(np.unique(adatanorm.obs['clusters-pc7'].astype(int)),adatanorm.uns['clusters-pc7_colors']))
#cl.apply(lambda x: colors[x])
cl=adatanorm.obs['clusters-pc7'].astype(int)

In [ ]:
import seaborn as sns
import matplotlib as plt
plt.pyplot.figure(figsize=(40,40))
plt.pyplot.scatter(re.X,re.Y,s=20,c='#FFFFFF',linewidths=0,edgecolors=None)
plt.pyplot.scatter(x=adatanorm.obs.X,y=adatanorm.obs.Y,c=cl.apply(lambda x: colors[x]),s=20,linewidths=0, edgecolors=None)
ax=plt.pyplot.gca()                            # get the axis
ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
ax.xaxis.tick_top()                     # and move the X-Axis      
ax.yaxis.tick_left() 

In [ ]:
#to create subclusters 
adatasub1=adatanorm[adatanorm.obs["clusters-pc7"].isin(["13"])]

In [ ]:
sc.pp.neighbors(adatasub1, n_neighbors=10, n_pcs=0)
sc.tl.leiden(adatasub1,resolution=0.8,key_added='Level2-cluster13')

In [ ]:
sc.tl.paga(adatasub1, groups='Level2-cluster13')
sc.pl.paga(adatasub1, color=['Level2-cluster13'])

In [ ]:
sc.tl.umap(adatasub1,min_dist=0.3)#,init_pos='paga')
sc.set_figure_params(scanpy=True, dpi=200)

In [ ]:
#to visualize subclusters of cluster 13
sc.pl.umap(adatasub1,color=['Level2-cluster13'],cmap='viridis')#,save='epith_clusters_0.svg')

In [ ]:
sc.pl.umap(adatasub1,color=adatasub1.var.index[0:74],cmap='viridis',vmax=10)

In [ ]:
sc.tl.rank_genes_groups(adatasub1, groupby='Level2-cluster13', method='wilcoxon')
sc.tl.dendrogram(adatasub1,groupby='Level2-cluster13')
sc.pl.rank_genes_groups_dotplot(adatasub1, n_genes=10, swap_axes=True)#,save='deg.pdf')

In [ ]:
sc.tl.rank_genes_groups(adatasub1,groupby='Position')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adatasub1)

In [ ]:
writer = pd.ExcelWriter('cluster13-Lev2.xlsx')
# write dataframe to excel
adatasub1.obs.to_excel(writer)
# save the excel
writer.save()